#Imports



In [ ]:
import numpy as np
import math
from scipy.optimize import linprog
import random
import matplotlib.pyplot as plt

In [ ]:
n_arms = 3
#n_arms = 2
#hospitals = 3
hospitals = 5

#Update UCB

In [ ]:
def new_ucb(r, N, t, c):
  ucb = []
  lcb = []
  #c = 0.4
  for i in range(len(r)):
    axu = []
    axl = []
    for j in range(len(r[0])):
      if N[i][j] == 0:
        axu.append(1)
        axl.append(0)
      else:
        axu.append(np.min([r[i][j]/N[i][j] + c*math.sqrt(math.log(t[i])/N[i][j]), 1]))
        axl.append(np.max([r[i][j]/N[i][j] - c*math.sqrt(math.log(t[i])/N[i][j]), 0]))
        #axu.append(np.min([r[i][j]/N[i][j] + 0.5*math.sqrt(math.log(t[i])/N[i][j]), 1]))
        #axl.append(np.max([r[i][j]/N[i][j] - 0.5*math.sqrt(math.log(t[i])/N[i][j]), 0]))
    ucb.append(axu)
    lcb.append(axl)
  return ucb, lcb

In [ ]:
def new_ucb1(r, N, t, c):
  ucb = []
  lcb = []
  #c = 0.4
  for i in range(len(r)):
    axu = []
    axl = []
    for j in range(len(r[0])):
      if N[i][j] == 0:
        axu.append(1)
        axl.append(0)
      else:
        axu.append(np.min([r[i][j]/N[i][j] + c[i]*math.sqrt(math.log(t[i])/N[i][j]), 1]))
        axl.append(np.max([r[i][j]/N[i][j] - c[i]*math.sqrt(math.log(t[i])/N[i][j]), 0]))
    ucb.append(axu)
    lcb.append(axl)
  return ucb, lcb

#Update Causal

In [ ]:
from scipy.optimize import linprog

def update_causal(r, N, t, r_h, N_h, t_h, ub, lb, c1):
  LCausal = []
  UCausal = []
  for i in range(n_arms):
    l = np.sum(np.array([r[k][i] for k in range(hospitals)]))/np.sum(np.array(t))
    u = 1 - (np.sum(np.array([N[k][i] for k in range(hospitals)])) - np.sum(np.array([r[k][i] for k in range(hospitals)])))/np.sum(np.array(t))
    LCausal.append(l)
    UCausal.append(u)
  causal = []
  for _ in range(hospitals):
    c = []
    for _ in range(n_arms):
      c.append(1)
    causal.append(c)

  for a in range(n_arms):
    c = []
    for _ in range(hospitals):
      c.append(0)
    for u in range(hospitals):
      c[u] = -1
      a1 = []
      a2 = []
      for i in range(hospitals):
        a1.append(-t[i]/np.sum(np.array(t)))
        a2.append(t[i]/np.sum(np.array(t)))
      A = [a1, a2]
      b = [-LCausal[a], UCausal[a]]
      bounds_of_x = []
      ub, lb = new_ucb1(r_h, N_h, t_h, c1)
      for h in range(hospitals):
        bounds_of_x.append((lb[h][a], ub[h][a]))
      res = linprog(c, A_ub=A, b_ub=b, bounds=bounds_of_x)
      #causal[u][a] = np.min([res.x[u], ub[u][a]])
      causal[u][a] = res.x[u]
  return causal

#Server

In [ ]:
def server_UCB(r, N, t, c):
  ucb = []
  lcb = []
  for j in range(len(r)):
    if N[j] == 0:
      ucb.append(1)
      lcb.append(0)
    else:
      ucb.append(np.min([r[j]/N[j] + c*math.sqrt(math.log(t)/N[j]), 1]))
      lcb.append(np.max([r[j]/N[j] - c*math.sqrt(math.log(t)/N[j]), 0]))
  return ucb, lcb

In [ ]:
def server(r, n, t, c, epsilon):
  ucb = []
  lcb = []
  est_reward = []
  for i in range(hospitals):
    ax = []
    for j in range(n_arms):
      if n[i][j] != 0:
        ax.append(r[i][j]/n[i][j])
      else:
        ax.append(1)
    est_reward.append(ax)
  r_total = []
  n_total = []
  t_total = []
  for i in range(hospitals):
    S_h = [i]
    for j in range(hospitals):
      ind_i = np.argmax(est_reward[i])
      arm_i = est_reward[i][ind_i]
      if i!=j:
        ind_j = np.argmax(est_reward[j])
        arm_j = est_reward[j][ind_j]
        if ind_i == ind_j and (arm_i - arm_j)**2 < epsilon:
          S_h.append(j)
    r_h = []
    n_h = []
    t_h = 0
    for j in range(n_arms):
      rew = 0
      num = 0
      for i in S_h:
        rew += r[i][j]
        num += n[i][j]
        t_h += n[i][j]
      r_h.append(rew)
      n_h.append(num)
    r_total.append(r_h)
    n_total.append(n_h)
    t_total.append(t_h)
    ucb_h, lcb_h = server_UCB(r_h, n_h, t_h, c)
    ucb.append(ucb_h)
    lcb.append(lcb_h)
  return ucb, lcb, r_total, n_total, t_total

#Train

In [ ]:
def train_adaptive(c, pi, responses, rewards, timesteps, bool_causal, bool_adaptive, epsilon = 0, c1 = 0):
  prob_explore = 0.5
  succ = []
  estimated_reward = []
  success = []
  times = []
  for i in range(hospitals):
    estimated_reward.append([])
    success.append([])
    succ.append(0)
    times.append(0)
  ucb_list = []
  lcb_list = []
  ucb = []
  lcb = []
  u_causal = []
  l_causal =  []
  u_causal_list = []
  l_causal_list = []
  #############
  u_caus_seq = []
  l_caus_seq = []
  ############
  regret = []
  r = []
  N = []
  t = []
  index = []
  reg = []
  ###############
  for i in range(hospitals):
    u = []
    l = []
    u_list = []
    l_list = []
    for j in range(n_arms):
      u.append([])
      l.append([])
      u_list.append([1])
      l_list.append([0])
    u_caus_seq.append(u)
    l_caus_seq.append(l)
    u_causal_list.append(u_list)
    l_causal_list.append(l_list)

  ###############
  for i in range(hospitals):
    u = []
    l = []
    u_list = []
    l_list = []
    for j in range(n_arms):
      u.append(1)
      l.append(0)
      u_list.append([1])
      l_list.append([0])
    ucb.append(u)
    lcb.append(l)
    ucb_list.append(u_list)
    lcb_list.append(l_list)
  for i in range(hospitals):
    u = []
    l = []
    for j in range(n_arms):
      u.append(1)
      l.append(0)
    u_causal.append(u)
    l_causal.append(l)
    regret.append([])
  for i in range(hospitals):
    l = []
    for j in range(n_arms):
      l.append(0)
    r.append(l)
  for i in range(hospitals):
    l = []
    for j in range(n_arms):
      l.append(0)
    N.append(l)
    t.append(0)
  for i in range(hospitals):
    l = []
    for j in range(n_arms):
      l.append(0)
    index.append(l)
    reg.append(0)
  for time in range(timesteps):
    clus = []
    for i in range(hospitals):
      clus.append(np.random.choice([1, 0], p = [pi[i], 1-pi[i]]))
    if time ==  0:
      ucb, lcb = new_ucb(r, N, t, c)
    for i in range(hospitals):
      for j in range(n_arms):
        ucb_list[i][j].append(ucb[i][j])
        lcb_list[i][j].append(lcb[i][j])
    if bool_causal:
      for i in range(hospitals):
        for j in range(n_arms):
          u_causal_list[i][j].append(u_causal[i][j])
          l_causal_list[i][j].append(l_causal[i][j])
    #print(ucb)
    for i in range(hospitals):
      if clus[i] == 1:
        if bool_causal:
          u_bound = [min(ucb[i][j], u_causal[i][j]) for j in range(n_arms)]
        else:
          u_bound = ucb[i]
        #arm = np.argmax(u_bound)
        ax = np.array(u_bound)
        arm = np.random.choice(np.where(ax == ax.max())[0])
        ######HERE
        if time < 1000:
          flag = np.random.choice([1, 0], p = [prob_explore, 1- prob_explore])
          if flag == 1:
            ##CHANGE WITH ARMS
            arm = np.random.choice([0, 1, 2])
            #arm = np.random.choice([0, 1])
        #if time % 1000 == 0 and i == 1:
        #  print(ucb[i])
        times[i] += 1
        if arm == np.argmax(rewards[i]):
          succ[i] += 1
        success[i].append(succ[i]/times[i])
        N[i][arm] += 1
        t[i] += 1
        prob = rewards[i][arm]
        #print(index)
        r[i][arm] += responses[i][arm][index[i][arm]]
        index[i][arm] += 1
        reg[i] += np.max(rewards[i]) - prob
        regret[i].append(reg[i])
        arm_now = np.argmax(rewards[i])
        if N[i][arm_now] != 0:
          estimated_reward[i].append(r[i][arm_now]/N[i][arm_now])
        else:
          estimated_reward[i].append(0)
###########################################
###########################################
    if time %100 == 0 and time != 0:
      prob_explore /= 1.5
    if bool_causal:
      ucb, lcb, r_h, n_h, t_h = server(r, N, t, c, epsilon)
      flag = True
      for i in range(hospitals):
        if t[i] == 0:
          flag = False
          break
      if np.sum(np.array(t)) % 100 == 0 and flag:
        bounds = []
        for i in range(hospitals):
          b = []
          for j in range(n_arms):
            b.append(min(u_causal[i][j], ucb[i][j]))
          bounds.append(b)
        u_causal = update_causal(r, N, t, r_h, n_h, t_h, ucb, lcb, c1)
        ###################
        for i in range(hospitals):
          for j in range(n_arms):
            u_caus_seq[i][j].append(u_causal[i][j])
            l_caus_seq[i][j].append(l_causal[i][j])
        #####################
    elif not bool_causal:
      if bool_adaptive:
        ucb, lcb, r_h, n_h, t_h = server(r, N, t, c, epsilon)
      else:
        ucb, lcb = new_ucb(r, N, t, c)
  #print(estimated_reward)
  #if bool_causal:
  #  print_bounds(rewards, u_caus_seq, l_caus_seq, "Causal")
  if bool_causal:
    return regret, ucb_list, lcb_list, u_causal_list, l_causal_list, estimated_reward, success
  else:
    return regret, ucb_list, lcb_list, estimated_reward, success